# 599 Capstone Project

This notebook connects to the NewAPI to return JSOn objects based on specific queries, loads specific elements from the object, then persists the data in an existing MySQL table.

## Resolve dependencies

In [1]:
! pip install newsapi-python

## Globally import libraries

Libraries needed mostly pertain to file access, dataframe manipulation, and NewsAPI call.

In [2]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime as dt
import re
import regex as rex
from collections import defaultdict, Counter
import random

from newsapi import NewsApiClient

# Set pandas global options
pd.options.display.max_rows = 17

## Connect to NewsAPI client

Pull masked key from environment variable.

In [3]:
api_key = os.environ['NewsAPIKey']

# Init APO connection
newsapi = NewsApiClient(api_key=api_key)

### Pull article info from API

Get source info in order to setup API call correctly. *Note:* Display turned off due to size of output.

### Define custom function for API call

Use the newsapi package `.get_everything()` call structure to retrieve articles based on a combination of key word query, source/domain, from/to dates, language (English-only), and number of pages returned (NewsAPI returns a max. of 100 articles per page). The results are appended to an empty list.

In [4]:
def news_api_urls(q=None,
                  s=None,
                  d_from='2023-05-01',
                  d_to='2023-05-31',
                  page=1,
                  api_lst=[]):
    '''Access API and pull content from resulting JSON object'''
    all_articles = newsapi.get_everything(q=q,
                                          domains=s,
                                          from_param=d_from,
                                          to=d_to,
                                          language='en',
                                          sort_by='popularity',
                                          page=page)

    #print(type(all_articles))
    #print(all_articles)
    #print('Article list: ', all_articles['articles'])
    #for article in all_articles['articles']:
        #print('Source ID:', article['source']['id'])
        #print('Source name:', article['source']['name'])
        #print('Author:', article['author'])
        #print('Title:', article['title'])
        #print('URL:', article['url'])
        #print('Publish date:', article['publishedAt'])
        #print('Article text:', article['content'], '\n')

    # Create a list of tuples from the dictionary data
    source_data01 = [(a['source']['name'],
                      a['author'],
                      a['title'],
                      a['url'],
                      a['publishedAt'],
                      a['content'])
                     for a in all_articles['articles']]

    api_lst.extend(source_data01)
    #print(api_lst)
    return(len(api_lst))

### Establish connection to API to access URLs

#### Set API filter parameters

The specific list of sources/domains was established to cast a wider net in order to maximize the diversity of content, while attempting to capture a combination of the most popular and/or mainstream online news source content (articles).

In [5]:
# Total API request grid: Sources x dates
#source_str = 'cnn, fox-news, the-wall-street-journal, abc-news, the-huffington-post, buzzfeed, breitbart-news, the-washington-post, usa-today, business-insider, nbc-news, msnbc, reuters, wired'
source_str = 'nypost.com, nytimes.com, msn.com, people.com, cnbc.com, forbes.com, vox.com'
#date_lst = ['2023-06-30', '2023-06-29', '2023-06-28', '2023-06-27', '2023-06-26',
#            '2023-06-25', '2023-06-24', '2023-06-23', '2023-06-22', '2023-06-21',
#            '2023-06-24', '2023-06-23', '2023-06-22', '2023-06-21', '2023-06-20',]

#date_lst = ['2023-06-19', '2023-06-18', '2023-06-17', '2023-06-16', '2023-06-15',
#            '2023-06-14', '2023-06-13', '2023-06-12', '2023-06-11', '2023-06-10',]

#date_lst = ['2023-06-09', '2023-06-08', '2023-06-07', '2023-06-06', '2023-06-05',
#            '2023-06-04', '2023-06-03', '2023-06-02', '2023-06-01', '2023-05-31',]

#date_lst = ['2023-05-30', '2023-05-29', '2023-05-28', '2023-05-27', '2023-05-26',
#            '2023-05-25', '2023-05-24', '2023-05-23', '2023-05-22', '2023-05-21',]

#date_lst = ['2023-05-20', '2023-05-19', '2023-05-18', '2023-05-17', '2023-05-16',
#            '2023-05-15', '2023-05-14', '2023-05-13', '2023-05-12', '2023-05-11',]

date_lst = ['2023-05-10', '2023-05-09', '2023-05-08', '2023-05-07', '2023-05-06',
            '2023-05-05', '2023-05-04', '2023-05-03', '2023-05-02', '2023-05-01',]

q_word_lst = []

### Access API

Run the custom function through a `for` loop to prevent over-loading calls to the API. For each date in `date_lst`, and for each page in the range of 1 to 10, the `news_api_urls` function will access the entered source/domains (as a string), appending the results to `api_lst`. A random sleep step was added to prevent rate limiting by the API.

In [6]:
'''Run individual request for each source/data/keyword combo
to maximize data scraped'''
api_record_lst01 = []

for d in date_lst:
    for p in range(1, 11):
        time.sleep(5 + 11 * random.random())
        try:
            lst_len = news_api_urls(s=source_str,
                                    d_from=d,
                                    d_to=d,
                                    page=p,
                                    api_lst=api_record_lst01)
        except:
            print(f'Page {p} is not available for {d}')
        print(f'Story Count: {lst_len}; ', end='')

Story Count: 100; Story Count: 200; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 288; Story Count: 388; Story Count: 488; Story Count: 588; Story Count: 688; Story Count: 788; Story Count: 830; Story Count: 830; Story Count: 830; Story Count: 830; Story Count: 830; Story Count: 930; Story Count: 1030; Story Count: 1130; Story Count: 1230; Story Count: 1330; Story Count: 1430; Story Count: 1477; Story Count: 1477; Story Count: 1477; Story Count: 1477; Story Count: 1577; Story Count: 1677; Story Count: 1777; Story Count: 1877; Story Count: 1977; Story Count: 2077; Story Count: 2157; Story Count: 2157; Story Count: 2157; Story Count: 2157; Story Count: 2257; Story Count: 2357; Story Count: 2457; Story Count: 2557; Story Count: 2657; Story Count: 2757; Story Count: 2835; Story Count: 2835; Story Count: 2835; Story Count: 2835; Story Count: 2935; Story Count: 3035; Story Count: 3135; Story Count: 3

Print size number of API records returned.

In [7]:
print(len(api_record_lst01))
#print(api_record_lst01)

5178


Convert the list to set in case there are duplicate URLs, the convert the set back to list type for use in MySQL transactions.

In [8]:
# Convert result list to set to eliminate duplicates
api_record_set01 = set(api_record_lst01)
#print(api_record_set01)
api_record_lst02 = list(api_record_set01)
#print(api_record_lst02)
print(len(api_record_lst02))

5043


## Initiate MySQL connection

Open connection to existing MySQL schema and table. Pull masked user and password from environment variable.

In [9]:
'''Set local environment variables to hide user name & password citation:
https://www.geeksforgeeks.org/how-to-hide-sensitive-credentials-using-python/
'''
user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='599_capstone')

# Create a cursor object
cursor = db_conn.cursor()

Display schema tables to ensure the connection was successful.

In [10]:
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

,Tables_in_599_capstone
0,nar_temp
1,news_articles


<class 'pandas.core.frame.DataFrame'>


### Establish logging policy

Configure logging so that connection errors can be reviewed. Logging output is sent to the pymysql.log file.

In [11]:
'''Logging citations:
1. https://chat.openai.com/share/7d376dad-eb91-40b7-b84f-55286fb29d35
2. https://docs.python.org/3/howto/logging.html#logging-basic-example
3. https://docs.python.org/3/howto/logging.html#logging-to-a-file
4. https://docs.python.org/3/howto/logging-cookbook.html#using-a-rotating-log-file-handler
5. https://docs.python.org/3/howto/logging-cookbook.html#using-a-timed-rotating-file-handler
'''

# Set up logging
logging.basicConfig(level=logging.INFO,
                    filename='pymysql.log',
                    filemode='a',
                    format='''>>>>>>>>>>>>>><<<<<<<<<<<<<<\n%(asctime)s - %(levelname)s - %(message)s''')

### Update individual tables

#### Update `news_articles` table from API

In [12]:
nat_tbl_name = 'nar_temp'
nwa_tbl_name = 'news_articles'

Load the list of API record results to a temporary MySQL table. Then update the main MySQL table by adding records based on a join with the temporary table. Using `title`, `publish_date`, and `author` as the join criteria ensures that only new URLs are added. The code block starts and ends with a wipe of the temp table. Also, the logging criteria has been embedded in order to output any errors generated.

In [13]:
'''Using cursor and loading into temp file:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/;
https://pynative.com/python-mysql-insert-data-into-database-table/
'''

# Execute query and measure execution time
start_time = time.time()

# Wipe temp table
try:
    nat_dlt_tble_stmnt = f"""DELETE FROM {nat_tbl_name}"""
    cursor.execute(nat_dlt_tble_stmnt)
    logging.info(f'''Successfully executed query:\n{nat_dlt_tble_stmnt}\n\nRecords scanned: {cursor.rowcount}''')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_dlt_tble_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'''Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n''')

# Execute query and measure execution time
start_time = time.time()

# Load data from CSV file into a temporary table
try:
    nat_csv_load_stmnt = f"""
    INSERT INTO {nat_tbl_name}
    (
    source_name,
    author,
    title,
    url,
    publish_date,
    content
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    """

    # Execute the query with multiple values
    cursor.executemany(nat_csv_load_stmnt, api_record_lst02)
    #cursor.execute(nat_csv_load_stmnt)
    logging.info(f'''Successfully executed query:\n{nat_csv_load_stmnt}\n\nRecords scanned: {cursor.rowcount}''')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_csv_load_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'''Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n''')

# Execute query and measure execution time
start_time = time.time()

# Insert new records into main table
try:
    nwa_load_stmnt = f"""
    INSERT INTO {nwa_tbl_name}
    (
    source_name,
    author,
    title,
    url,
    publish_date,
    content
    )
    SELECT
        tp.source_name,
        tp.author,
        tp.title,
        tp.url,
        tp.publish_date,
        tp.content
    FROM {nat_tbl_name} AS tp
    LEFT JOIN {nwa_tbl_name} AS mn
    ON tp.title = mn.title
        AND CAST(LEFT(tp.publish_date,10) AS DATE)=CAST(LEFT(mn.publish_date,10) AS DATE)
        AND tp.author = mn.author
    """
    cursor.execute(nwa_load_stmnt)
    logging.info(f'''Successfully executed query:\n{nwa_load_stmnt}\n\nRecords scanned: {cursor.rowcount}''')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nwa_load_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'''Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n''')

# Execute query and measure execution time
start_time = time.time()

# Wipe temp table
try:
    cursor.execute(nat_dlt_tble_stmnt)
    logging.info(f'''Successfully executed query:\n{nat_dlt_tble_stmnt}\n\nRecords scanned: {cursor.rowcount}''')
except mysql.Error as e:
    logging.error(f'Error executing query:\n{nat_dlt_tble_stmnt}\n\n{e}')
finally:
    end_time = time.time()
    logging.info(f'''Time taken: {end_time - start_time:.3f} seconds\n>>>>>>>>>>>>>><<<<<<<<<<<<<<\n\n''')

### Commit changes and close cursor and connection instances

This step is integral to committing any remaining MySQL calls, as well as releasing the connection.

In [14]:
# Commit the changes to the database
db_conn.commit()

# Close the cursor and database connection
cursor.close()
db_conn.close()